# Save and restore models

* [source site](https://www.tensorflow.org/beta/tutorials/keras/save_and_restore_models)

In [1]:
# !pip install -q h5py pyyaml
# if you already install, you can delete

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

In [3]:
tf.__version__

'2.0.0-beta1'

In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0
# for tutorial used just 1000 samples

In [5]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model


# Create a basic model instance
model = create_model()
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [10]:
model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

W0626 20:48:12.083481 139957589088064 deprecation.py:323] From /home/hyunsu/anaconda3/envs/tf20_py36/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
  32/1000 [..............................] - ETA: 6s - loss: 2.3210 - accuracy: 0.1250
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 430us/sample - loss: 1.1004 - accuracy: 0.6940 - val_loss: 0.7201 - val_accuracy: 0.7750
Epoch 2/10
  32/1000 [..............................] - ETA: 0s - loss: 0.2810 - accuracy: 0.9375
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 73us/sample - loss: 0.4093 - accuracy: 0.8810 - val_loss: 0.5531 - val_accuracy: 0.8260
Epoch 3/10
  32/1000 [..............................] - ETA: 0s - loss: 0.3302 - accuracy: 0.9062
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 74us/sample - loss: 0.2975 - accuracy: 0.9140 - val_loss: 0.4653 - val_accuracy: 0.8460
Epoch 4/10
  32/1000 [..............................] - ETA: 0s - loss: 0.2187 - accuracy: 0.9375
Epoch 0

In [11]:
!ls {checkpoint_dir}

checkpoint		     cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002  cp.ckpt.index


In [12]:
!ls training_1/

checkpoint		     cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002  cp.ckpt.index


In [13]:
model = create_model() # rebuil model for load pretrained model

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 68us/sample - loss: 2.3763 - accuracy: 0.0710
Untrained model, accuracy:  7.10%


In [14]:
model.load_weights(checkpoint_path) # load the weight variables from checkpoint dir
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 22us/sample - loss: 0.4102 - accuracy: 0.8670
Restored model, accuracy: 86.70%


In [15]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)

W0626 20:53:27.566112 139957589088064 callbacks.py:859] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.



Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [16]:
!ls training_2

checkpoint			  cp-0025.ckpt.data-00001-of-00002
cp-0000.ckpt.data-00000-of-00002  cp-0025.ckpt.index
cp-0000.ckpt.data-00001-of-00002  cp-0030.ckpt.data-00000-of-00002
cp-0000.ckpt.index		  cp-0030.ckpt.data-00001-of-00002
cp-0005.ckpt.data-00000-of-00002  cp-0030.ckpt.index
cp-0005.ckpt.data-00001-of-00002  cp-0035.ckpt.data-00000-of-00002
cp-0005.ckpt.index		  cp-0035.ckpt.data-00001-of-00002
cp-0010.ckpt.data-00000-of-00002  cp-0035.ckpt.index
cp-0010.ckpt.data-00001-of-00002  cp-0040.ckpt.data-00000-of-00002
cp-0010.ckpt.index		  cp-0040.ckpt.data-00001-of-00002
cp-0015.ckpt.data-00000-of-00002  cp-0040.ckpt.index
cp-0015.ckpt.data-00001-of-00002  cp-0045.ckpt.data-00000-of-00002
cp-0015.ckpt.index		  cp-0045.ckpt.data-00001-of-00002
cp-0020.ckpt.data-00000-of-00002  cp-0045.ckpt.index
cp-0020.ckpt.data-00001-of-00002  cp-0050.ckpt.data-00000-of-00002
cp-0020.ckpt.index		  cp-0050.ckpt.data-00001-of-00002
cp-0025.ckpt.data-00000-of-00002  cp-0050.ckpt.index


In [32]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [33]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 67us/sample - loss: 0.4637 - accuracy: 0.8770
Restored model, accuracy: 87.70%


### save trained weights manually.

In [34]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model() # build untrained model
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

W0626 21:13:52.325988 139957589088064 util.py:244] Unresolved object in checkpoint: (root).optimizer.iter
W0626 21:13:52.326528 139957589088064 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_1
W0626 21:13:52.326868 139957589088064 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_2
W0626 21:13:52.327120 139957589088064 util.py:244] Unresolved object in checkpoint: (root).optimizer.decay
W0626 21:13:52.327360 139957589088064 util.py:244] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0626 21:13:52.327621 139957589088064 util.py:252] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/alpha/guide/checkpoints#

1000/1000 [==============================] - 0s 67us/sample - loss: 0.4637 - accuracy: 0.8770
Restored model, accuracy: 87.70%


In [35]:
!ls checkpoints/

checkpoint			   my_checkpoint.data-00001-of-00002
my_checkpoint.data-00000-of-00002  my_checkpoint.index


### save whole model including the layer archtecture

In [36]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 160us/sample - loss: 1.1351 - accuracy: 0.6790
Epoch 2/5
1000/1000 [==============================] - 0s 40us/sample - loss: 0.4094 - accuracy: 0.8960
Epoch 3/5
1000/1000 [==============================] - 0s 39us/sample - loss: 0.2798 - accuracy: 0.9310
Epoch 4/5
1000/1000 [==============================] - 0s 39us/sample - loss: 0.1919 - accuracy: 0.9580
Epoch 5/5
1000/1000 [==============================] - 0s 39us/sample - loss: 0.1511 - accuracy: 0.9710


In [37]:
# build same model 
new_model = keras.models.load_model('my_model.h5')
new_model.summary()


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [38]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 66us/sample - loss: 0.4422 - accuracy: 0.8590
Restored model, accuracy: 85.90%


* following contents of the source site, said that the function is experimental
* so it may change in future version, thus, didn't replicate here.